In [48]:
import pandas as pd 
import yfinance as yf 

class YahooFin:

    def __init__(self, ticker):
        self.historical_stock_price = None
        self.ticker = ticker
        self.price = None
        self.start_date = "2019-01-01"
        self.end_date = "2022-12-31"

    # fetching past data from the yahoo finance
    def get_data(self):
        try:
            if type(self.ticker)!=list:
                self.ticker = [self.ticker]

            self.historical_stock_price = yf.download(self.ticker, start=self.start_date, end=self.end_date, interval="1d")
            self.price = self.historical_stock_price['Adj Close']
            return self.price
            # self.price.index = pd.to_datetime(self.historical_stock_price.index, format="%y%m%d")
        except Exception as err:
            print(err)      

In [114]:
"""
The BL model takes a Bayesian approach to asset allocation. It combines a prior estimate of return with views on certain assets, to produce a 
posterior estimate of the expected returns.

Advantage of BL:

1. you can provide views on only a subset of assets and BL will propagate, taking into account the covariance with other assets.
2. you can provide confidence in your views
3. BL posterior returns results in much more stable portfolio than using mean historical return.


Black Litterman Allocation
https://pyportfolioopt.readthedocs.io/en/latest/BlackLitterman.html

Interactive BL implementation by Thomas Kirschenman
https://github.com/thk3421-models/cardiel
"""
import pypfopt
from pypfopt import black_litterman, risk_models, objective_functions
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import DiscreteAllocation

# from Collector import YahooFin
import yfinance as yf


class BlackLitterman:

    def __init__(self):
        self.delta = None
        self.prior = None
        self.mcaps = {}
        self.cov_shrinkage = None
        self.price = None # List of tickers to create a portfolio
        self.market_price = None # Any comparision ticker like S&P to refer the market return
        self.omega = None
        self.return_bl = None
        self.alloc = None
        self.weight = None
        self.viewdict = None
        self.cov_mat_bl = None

    def get_market_price(self):
        return self.market_price

    def get_stock_price(self):
        return self.price

    def initialize_variables(self):
        ticker = ["MSFT", "AMZN", "NAT", "BAC", "DPZ", "DIS", "KO", "MCD", "COST", "SBUX"]
        y_price = YahooFin(ticker=["MSFT", "AMZN", "NAT", "BAC", "DPZ", "DIS", "KO", "MCD", "COST", "SBUX"])
        self.price = y_price.get_data()
        

        y_market_price = YahooFin(ticker=["SPY"])
        self.market_price = y_market_price.get_data()

        for t in ticker:
            s = yf.Ticker(t)
            self.mcaps[t] = s.info["marketCap"]

        self.cov_shrinkage = risk_models.CovarianceShrinkage(self.price).ledoit_wolf()
        self.delta = black_litterman.market_implied_risk_aversion(self.market_price)


    def evaluate_prior(self):

        """
            prior: N x 1 vector of prior expected return (by definition)
            Prior is quantified by the market-implied risk premium, which is the market's excess return divided by its variance:
            delta = ()
        """
        
        self.prior = black_litterman.market_implied_prior_returns(self.mcaps, self.delta, self.cov_shrinkage)

        # plot a brah 
        # market_prior.plot.barh(figsize=(10,5));

    def evaluate_view(self):
        """
        NOTE: We are using a absolute view for now. This will be replaced by the self.weighted metrics derived from
        the tuned FinLLM 

        viewdict = <call-back function>
        confidence = <call-back function>(viewdict)
        """
        self.viewdict = {
            "AMZN": 0.10,
            "BAC": 0.30,
            "COST": 0.05,
            "DIS": 0.05,
            "DPZ": 0.20,
            "KO": -0.05,  # 5% down
            "MCD": 0.15,
            "MSFT": 0.10,
            "NAT": 0.50, 
            "SBUX": 0.10
        }

        # Either use confidence to evaluate omega or evaluate uncertainity matrix omega by specifying 1 standard deviation
        # conidence intervals, i.e bounds which we think will containthe true return 68% of the time. This is better
        # than putting arbitary percentage .

        #----------------------------------------------------------------------------------------------------------
        # confidence =  [
        #     0.6,
        #     0.4,
        #     0.2,
        #     0.5,
        #     0.7, # confident in dominos
        #     0.7, # confident KO will do poorly
        #     0.7, 
        #     0.5,
        #     0.1,
        #     0.4
        # ]

        # bl = BlackLittermanModel(self.cov_shrinkage, pi=self.prior, absolute_views=viewdict, omega="idzorek", view_confidences=confidence)

        # self.omega = bl.omega

        # OR -------------------------------------------------------------------------------------------------------

        intervals = [
            (0, 0.25),
            (0.1, 0.4),
            (-0.1, 0.15),
            (-0.05, 0.1),
            (0.15, 0.25),
            (-0.1, 0),
            (0.1, 0.2),
            (0.08, 0.12),
            (0.1, 0.9),
            (0, 0.3)
        ]

        variances = []
        for lb, ub in intervals:
            sigma = (ub - lb)/2
            variances.append(sigma ** 2)

        self.omega = np.diag(variances)

    def evaluate_posterior(self):

        bl = BlackLittermanModel(self.cov_shrinkage, pi="market", market_caps=self.mcaps, risk_aversion=self.delta,\
                                    absolute_views=self.viewdict, omega=self.omega)
        # Posterior estimate of returns
        self.return_bl = bl.bl_returns()
        return_df = pd.DataFrame([self.prior, self.return_bl, pd.Series(self.viewdict)], index=["Prior", "Posterior", "Views"]).T

        # rets_df.plot.bar(figsize=(12,8));

        self.cov_mat_bl = bl.bl_cov()

    def portfolio_allocation(self):

        ef = EfficientFrontier(self.return_bl, self.cov_mat_bl)
        ef.add_objective(objective_functions.L2_reg)
        ef.max_sharpe()
        self.weight = ef.clean_weights()
        # pd.Series(self.weight).plot.pie(figsize=(10,10));
        da = DiscreteAllocation(self.weight, self.price.iloc[-1], total_portfolio_value=20000)
        self.alloc, leftover = da.lp_portfolio()
        print(f"Leftover: ${leftover:.2f}")


In [115]:
"""
low
low to moderate
moderate
moderate high
high
very high
"""

riskometer = {"low":0, "low_to_moderate":1, "moderate":2, "moderate high":3, "high":4, "very_high":5}
port_class = {"low":[], "low_to_moderate":[], "moderate":[], "moderate high":[], "high":[], "very_high":[]}

stock_universe = ['AAPL','AMGN','AMZN','AXP','BA','CAT','CRM','CSCO','CVX','DIS','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','KO','MCD','MMM','MRK','MSFT','NKE','PG','TRV','UNH','V','VZ','WMT']

In [116]:
"Source of info: https://www.bloomberg.com/markets/rates-bonds/government-bonds/us"
treasury_yield_rate = {"3_month":4.63, "6_month":4.52, "12_month":4.30, "2_year":4.10, "5_year":4.06, "10_year":4.24,"30_year":4.5}

In [117]:
b = BlackLitterman()

In [118]:
b.initialize_variables()

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed


In [119]:
b.evaluate_prior()

In [120]:
b.evaluate_view()

In [121]:
b.evaluate_posterior()

In [122]:
b.portfolio_allocation()

Leftover: $0.90


/home/diti/miniconda3/envs/devbox/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


In [123]:
b.alloc

{'AMZN': 35,
 'BAC': 68,
 'COST': 2,
 'DIS': 13,
 'DPZ': 12,
 'MCD': 8,
 'MSFT': 9,
 'NAT': 1062,
 'SBUX': 22}